# Review

- OpenAPI 
    - 네이버API
    - 공공데이타포탈(data.go.kr)
    - 기상청 
    - 통계청 
- 작업과정 
    - 1) 서비스 요청(requests, urllib)
    - 2) json, xml => 파싱 => 2차원리스트, 딕셔너리 
    - 3) 판다스 데이타프레임 구조 => csv, 엑셀, DB(sqlite, mySQL, 오라클)

- XML 
    - *.xml 
    - RSS 
    - 부산광역시_부산명소정보 서비스

# 부산광역시_부산명소정보 서비스
- 부산관광명소의 이름, 주소, 홈페이지, 전화번호, 좌표, 상세정보를 조회하는 서비스

- 1) 서비스 신청
    - https://www.data.go.kr/data/15063481/openapi.do
    - 키 정보 확인 
    :	
Yhq6n9vlpaKZsCOw0jec8zIdZ8p+Bpuku6WLjVgDhqXRW6dHnfXoauSEj19jpnjv59CUGspyTxVCgCeXOua7dg==

- 2) 레퍼런스 및 샘플 코드 확인 
    - https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15063481

### 3) 데이타 요청

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [35]:
url = 'http://apis.data.go.kr/6260000/AttractionService/getAttractionKr'
myKey = 'Yhq6n9vlpaKZsCOw0jec8zIdZ8p+Bpuku6WLjVgDhqXRW6dHnfXoauSEj19jpnjv59CUGspyTxVCgCeXOua7dg=='
params ={'serviceKey' : myKey, 
         'pageNo' : '1', 
         'numOfRows' : '30', 
         #'resultType' : '', 
         # 'UC_SEQ' : '' 
        }

response = requests.get(url, params=params)
print(response)
print(response.text)

<Response [200]>
<?xml version="1.0" encoding="UTF-8"?><response>
  <header>
    <resultCode>00</resultCode>
    <resultMsg>NORMAL_CODE</resultMsg>
  </header>
  <body>
    <items>
      <item>
        <MAIN_TITLE>흰여울문화마을</MAIN_TITLE>
        <LNG>129.04402</LNG>
        <MIDDLE_SIZE_RM1>점보버스 휠체어 리프트 장착 차량 운행, 요금 할인 50%, 장애인 주차장(절영해안산책로 공영주차장, 유료)</MIDDLE_SIZE_RM1>
        <UC_SEQ>255</UC_SEQ>
        <USAGE_AMOUNT>무료</USAGE_AMOUNT>
        <CNTCT_TEL>051-419-4067</CNTCT_TEL>
        <MAIN_IMG_NORMAL>https://www.visitbusan.net/uploadImgs/files/cntnts/20191222164810529_ttiel</MAIN_IMG_NORMAL>
        <TRFC_INFO>도시철도 1호선 남포역 6번 출구 → 7, 71, 508 버스 환승 → 영선동 백련사 하차
버스 7,71,508 영선동 백련사 하차
부산시티투어버스 부산역(점보버스) → 흰여울문화마을 하차
주차 절영해안산책로 입구 공영 주차장 (유료)</TRFC_INFO>
        <HLDY_INFO/>
        <ITEMCNTNTS>절영해안산책로 가파른 담벼락 위로 독특한 마을 풍경이 보인다. 해안가 절벽 끝에 바다를 따라 난 좁은 골목길 안쪽으로 작은 집들이 다닥다닥 붙어 있다. 피난민들의 애잔한 삶이 시작된 곳이자 현재는 마을주민과 함께하는 문화마을공동체 흰여울문화마을이 자리한 곳이다.
부산의 대표적인 원도심 흰여울길은 봉래산 기슭에서 굽이쳐 내리는 물줄기가 마치 흰 눈이 내

### 4) 파싱 => 데이타 추출 => 2차원 데이타

In [36]:
# 숩객체화 
soup = BeautifulSoup(response.text, 'xml')
# print(soup)
target_list = soup.find_all('item')
print(len(target_list))
# print(target_list[0])

30


In [37]:
# 정보 추출 (1개요소)
title = target_list[0].MAIN_TITLE.text
addr = target_list[0].ADDR1.text
usage = target_list[0].USAGE_AMOUNT.text
trfc = target_list[0].TRFC_INFO.text
homepage = target_list[0].HOMEPAGE_URL.text
print(title)
print(addr)
print(usage)
print(trfc)
print(homepage)

흰여울문화마을
부산광역시 영도구 흰여울길
무료
도시철도 1호선 남포역 6번 출구 → 7, 71, 508 버스 환승 → 영선동 백련사 하차
버스 7,71,508 영선동 백련사 하차
부산시티투어버스 부산역(점보버스) → 흰여울문화마을 하차
주차 절영해안산책로 입구 공영 주차장 (유료)
http://www.ydculture.com/huinnyeoulculturetown/


In [29]:
# print(target_list[0])

In [38]:
# 테이블 구조로 변경 
result_list = []
for target in target_list:
    title = target.MAIN_TITLE.text
    addr = target.ADDR1.text
    usage = target.USAGE_AMOUNT.text
    trfc = target.TRFC_INFO.text
    homepage = target.HOMEPAGE_URL.text
    result_list.append([title, addr, usage, trfc, homepage])

print(result_list[-1])

['아미동 비석마을', '부산광역시 서구 아미로49', '무료', '도시철도 1호선 토성역 8번 출구 도보 15분\n마을버스 사하구1-1 서구2 산상교회(비석문화마을) 하차\n주차 아미골 공영주차장', '']


### 데이타프레임 => csv 파일

In [39]:
df = pd.DataFrame(result_list, 
            columns=['제목', '주소', '요금', '교통정보', '홈페이지'])
# df

In [44]:
df.to_csv?

In [43]:
df.to_csv('output/부산명소.csv', 
          index=False, encoding='cp949', 
          errors='ignore')

# 데이타베이스 

- 데이타이베이스(*.db, *.sqlite. *.sqlite3)
- 테이블
- 행(레코드), 열(속성, 컬럼)

# SQL 
- Create
- Read
- Update 
- Delete 

### 샘플 데이타 베이스 파일 준비 

- pyclass/data/test.db 확인

# SQL 
- 대소문자를 구별하지 않는다 
- 여러행으로 명령어를 입력해도 된다. 
- 명령어 끝에는 세미콜른(;)으로 종료 